In [ ]:
import importlib
import codecs
import os
import pandas
import numpy 
import datetime as dt
import scipy.stats
import pandasql.sqldf
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pandas.options.display.max_rows = 200
pandas.options.display.max_columns = 50
pandas.options.display.width = 300
pandas.set_option('display.float_format', lambda x: '%.8f' % x)

import trm3.impute
import trm3.utility

In [406]:
rows=3000
letters=['a','b','c','d', numpy.NaN]
df = pandas.DataFrame({'A' : numpy.trunc(3*numpy.random.random(rows))+1,
                       'B' : numpy.trunc(3*numpy.random.random(rows))+1,
                       'C' : numpy.random.random(rows),
                       'D' : numpy.random.random(rows),
                       'E' : numpy.random.random(rows),
                       'F' : numpy.random.random(rows),
                       'G' : numpy.random.random(rows),
                       'X' : numpy.random.random(rows),
                       'M' : numpy.NaN*numpy.random.random(rows)
                      })

df['C'] = df['C'].apply(lambda x: x if x>=.2 else numpy.NaN)
df['D'] = df['D'].apply(lambda x: x if x>=.1 else numpy.NaN)
df['E'] = df['E'].apply(lambda x: 1 if x>=.5 else numpy.NaN)
df['F'] = df['F'].apply(lambda x: letters[int(5*x)])
df['G'] = df['G'].apply(lambda x: 'ABX' if x>=.5 else numpy.NaN)
df


,A,B,C,D,E,F,G,M,X
0,3.00000000,2.00000000,0.47043270,0.68565772,1.00000000,c,NaN,nan,0.78560740
1,3.00000000,3.00000000,0.63521607,0.87536287,nan,d,ABX,nan,0.32124124
2,1.00000000,1.00000000,0.74589148,0.34181687,nan,c,ABX,nan,0.10116874
3,3.00000000,2.00000000,0.32608369,0.64173490,1.00000000,NaN,ABX,nan,0.83554825
4,2.00000000,3.00000000,0.33695895,0.31686655,nan,c,ABX,nan,0.27323429
5,1.00000000,2.00000000,0.56997031,0.57040882,nan,b,NaN,nan,0.13272273
6,2.00000000,2.00000000,nan,0.41685108,1.00000000,d,ABX,nan,0.62720746
7,2.00000000,2.00000000,0.32744654,nan,nan,NaN,ABX,nan,0.35409748
8,3.00000000,2.00000000,0.26151082,0.94134751,nan,c,ABX,nan,0.76770475
9,1.00000000,1.00000000,0.42554167,0.19892102,1.00000000,NaN,NaN,nan,0.31661684


In [400]:
df.groupby(["A"])["F"].agg(lambda x: pandas.Series.mode(x).tolist()[0]) 
df.groupby(["A"])["D"].agg(lambda x: scipy.stats.mode(x)[0][0])

A
1.00000000    b
2.00000000    c
3.00000000    d
Name: F, dtype: object

A
1.00000000   -3.54985216
2.00000000   -3.07417325
3.00000000   -3.42101816
Name: D, dtype: float64

In [407]:
df["M"].nunique()

0

In [404]:
importlib.reload(trm3.impute)    
my_imputer = trm3.impute.Imputer(train_df = df,
                                 test_df = None,
                                 columns = ["C", "D", "E", "F", "G"],
                                 groupby = ["A","B"],
                                 label = "X",
                                 method = 4,
                                 mode_fallback_distinct_values = 4,
                                 mean_label_min_obs = .02,
                                 mean_label_bins = 10,
                                 nominal_new_category_min_obs = .005,
                                 missing_indicators = False,
                                 missing_indicators_min_obs = .005,
                                 model_x_columns = None                          
                                )

my_imputer_rules = my_imputer.fit()
my_imputer_rules.impute(impute_df=df, inplace=False)

<module 'trm3.impute' from 'D:\\projects\\Python36-Package-Development\\trm3pylib\\trm3\\impute.py'>

DEBUG:  1 - C



                    select a.*, 
                           cast(max(0.02*b.z__nobs,2) as integer) as z__min_label_min_obs
                    from (select A, B, z__bins
                                 ,avg(X) as z__bin_avgresp
                                 ,count(*) as z__bin_nobs
                                 ,avg(C) as z__bin_col_mean
                          from tdf
                          where C is not NULL 
                          group by A, B, z__bins
                          ) a 
                    join (select A, B, z__nobs
                          from gbdf                                              
                          ) b
                       on a.A = b.A and a.B = b.B
                    order by A, B, z__bins
                    
[C] -> 0.16365885734558105 seconds
DEBUG:  2 - D



                    select a.*, 
                           cast(max(0.02*b.z__nobs,2) as integer) as z__min_label_min_obs
                    from (select A, B, z__bins
                                 ,avg(X) as z__bin_avgresp
                                 ,count(*) as z__bin_nobs
                                 ,avg(D) as z__bin_col_mean
                          from tdf
                          where D is not NULL 
                          group by A, B, z__bins
                          ) a 
                    join (select A, B, z__nobs
                          from gbdf                                              
                          ) b
                       on a.A = b.A and a.B = b.B
                    order by A, B, z__bins
                    
[D] -> 0.16216039657592773 seconds
DEBUG:  3 - E



                    select a.*, 
                           cast(max(0.02*b.z__nobs,2) as integer) as z__min_label_min_obs
                    from (select A, B, z__bins
                                 ,avg(X) as z__bin_avgresp
                                 ,count(*) as z__bin_nobs
                                 ,avg(E) as z__bin_col_mean
                          from tdf
                          where E is not NULL 
                          group by A, B, z__bins
                          ) a 
                    join (select A, B, z__nobs
                          from gbdf                                              
                          ) b
                       on a.A = b.A and a.B = b.B
                    order by A, B, z__bins
                    
[E] -> 0.13713335990905762 seconds
DEBUG:  4 - F
[F] -> 0.029027938842773438 seconds
DEBUG:  5 - G
[G] -> 0.030579328536987305 seconds


,A,B,C,D,E,F,G,X
0,2.00000000,1.00000000,0.69071761,0.24694871,1.00000000,a,[nan],0.91980072
1,1.00000000,3.00000000,0.64465929,0.80242503,0.00000000,a,ABX,0.19358655
2,3.00000000,2.00000000,0.94080550,0.67464362,0.00000000,d,ABX,0.99118040
3,1.00000000,2.00000000,0.30468801,0.43125731,1.00000000,c,[nan],0.71013446
4,1.00000000,3.00000000,0.41344213,0.45895080,0.00000000,b,[nan],0.04778466
5,1.00000000,1.00000000,0.66364175,0.90242035,0.00000000,b,ABX,0.22583322
6,3.00000000,3.00000000,0.23062369,0.50925337,1.00000000,a,ABX,0.13170590
7,2.00000000,1.00000000,0.92982517,0.13734665,1.00000000,a,[nan],0.10093831
8,2.00000000,2.00000000,0.70113235,0.56685208,0.00000000,a,ABX,0.01092097
9,3.00000000,2.00000000,0.44390469,0.63642418,0.00000000,b,ABX,0.73728193


In [362]:
my_imputer_rules.rules_list[2][3]


z__method     z__nmiss  z__nobs  z__unique z__column   z__df_nmiss  z__df_nobs  z__df_unique  z__df_mode z__column_type  z__df_mean    z__mean    z__mode  z__df_median  z__median  z__indicator  z__missing_sumresp  z__missing_avgresp  index z__bins  z__bin_avgresp  \
A          B                                                                                                                                                                                                                                                                                     
1.00000000 1.00000000          4 162.00000000      138          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         83.44819479          0.51511231      0    None      0.48427197   
           2.00000000          4 151.00000000      185          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         76.03464769          0.50354071      1    None      0.47262456   
           3.00000000          4 156.00000000      149          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         78.54086895          0.50346711      2    None      0.49797177   
2.00000000 1.00000000          4 169.00000000      180          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         90.22577394          0.53388032      3    None      0.51101006   
           2.00000000          4 179.00000000      169          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         85.54915393          0.47792823      4    None      0.53002175   
           3.00000000          4 177.00000000      144          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         93.78938197          0.52988351      5    None      0.48398695   
3.00000000 1.00000000          4 182.00000000      174          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         94.50004332          0.51923101      6    None      0.53082133   
           2.00000000          4 185.00000000      175          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         95.05589227          0.51381563      7    None      0.47103814   
           3.00000000          4 168.00000000      157          1         E 1529.00000000        1471             1  1.00000000        Numeric  1.00000000 1.00000000 1.00000000    1.00000000 1.00000000         False         87.26823550          0.51945378      8    None      0.48659741   

                       z__bin_nobs  z__bin_col_mean  z__min_label_min_obs  z__bin_col_median    z__diff  z__adj_diff  z__adj_diff_rank  z__impute z__impute_type  
A          B                                                                                                                                                      
1.00000000 1.00000000          138       1.00000000                     2                nan 0.03084034   0.03084034        1.00000000          0    binary zero  
           2.00000000          185       1.00000000                     3                nan 0.03091615   0.03091615        1.00000000          0    binary zero  
           3.00000000          149       1.00000000                     2                nan 0.00549534   0.00549534        1.00000000          0    binary zero  
2.00000000 1.0000

In [ ]:
from collections import namedtuple
Point = namedtuple('Point', 'x y')
pt1 = Point(1.0, 5.0)
pt2 = Point(2.5, 1.5)
w = pt1._asdict()
w['x']

In [ ]:
q='''select A, B, F, sum(case when F is not null then 1 else 0 end) as nobs
from df
group by A, B, F
order by A, B, nobs desc
  '''

pandasql.sqldf(q).groupby(["A", "B"])["F"].first()


In [ ]:
df.dtypes

In [ ]:
tdf.groupby([gbcolumns])[col]. \
                    transform(lambda x: pandas.qcut(x, self.__mean_label_bins, labels=False, duplicates="drop")).astype('float64')

In [ ]:
x = {"A": 1,
     "B": 2,
     "C": 3,
     "D": 4}

print(x)

In [ ]:
for key in x:
    print(key + " - " + str(x[key]) + "\n")

In [ ]:
class myclass:
    def __init__(self,a): 
        self.a = a

    @property
    def a(self):
        return self.__a

    @a.setter
    def a(self, x):
        self.__a = x
        
    def test(self, zdf):
        _lv = self.__a
        # zdf.eval("X = A * @_lv", inplace=True)
        zdf["X"] = zdf["A"]*self.__a
        
p = myclass(5)
p.a
p.test(df)
df

In [ ]:
print(str(["parameter","Message explaining the error which can be long in some cases"]))

In [ ]:
code += x_indent(2) + x_and(cond, line) + "x['" + gbcol + "'] == " + x_val(_groupby_values[i][gc])

In [ ]:
def x_and(condition, code_line) -> str:
    if condition > 1:
        return "and "
    elif code_line == 1:
        return "if "
    else:
        return "elif "

def x_val(x) -> str:
    if type(x) == str:
        return "'" + x + "'"
    else:
        return str(x)

def x_indent(n):
    return " " * n * 4

In [ ]:
code = '''
def impute_calc(x):
    if type(x['C']) == numpy.NaN:
        if x['A'] == 1.0 and x['B'] == 1.0:
            return 0.729783375567
        elif x['A'] == 1.0 and x['B'] == 1.0:
            return 0.729783375567
        elif x['A'] == 1.0 and x['B'] == 2.0:
            return 0.577250026067
        elif x['A'] == 1.0 and x['B'] == 3.0:
            return 0.585054851176
        elif x['A'] == 2.0 and x['B'] == 1.0:
            return 0.595745017994
        elif x['A'] == 2.0 and x['B'] == 2.0:
            return 0.580186070545
        elif x['A'] == 2.0 and x['B'] == 3.0:
            return 0.584361868687
        elif x['A'] == 3.0 and x['B'] == 1.0:
            return 0.586579718828
        elif x['A'] == 3.0 and x['B'] == 2.0:
            return 0.592148803494
        elif x['A'] == 3.0 and x['B'] == 3.0:
            return 0.588026309637
        else:
            return 0.614602717576
    else:
        return x['C']
        '''
exec(code)
z = df.apply(impute_calc, axis=1)
z

In [364]:
a = [[1,2],[3,4]]
for x, y in a:
    print(str(x) + " - " + str(y))


1 - 2
3 - 4


In [ ]:
if pandas.isnull(z[5]):
    print("True")

In [357]:
q = "Number %s is not the same as" % 1
q

'Number 1 is not the same as'